In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
cd /content/drive/My Drive/Cloab Notebooks/matrix/matrix_two/dw_matrix_car/

/content/drive/My Drive/Cloab Notebooks/matrix/matrix_two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT  = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  
  factorize_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat: 
   df[feat] = factorize_values
  else:
   df[feat + SUFFIX_CAT] = factorize_values

In [8]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [18]:
X = df[cat_feats].values
y = df['price_value'].values


model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X,y, cv=3, scoring='neg_mean_absolute_error' )
np.mean(scores)

-19747.81093847179

In [0]:
def run_model(model, feats):
 X = df[feats].values
 y = df['price_value'].values
 scores = cross_val_score(model, X,y, cv=3, scoring='neg_mean_absolute_error' )
 return np.mean(scores), np.std(scores)

## DecisionTree

In [20]:
run_model( DecisionTreeRegressor(max_depth=5) , cat_feats)

(-19747.810938471786, 75.95212671816479)

## Random Forest

In [21]:
model =  RandomForestRegressor(max_depth=5, n_estimators=50 , random_state=0)
run_model(model, cat_feats )

(-18864.27843495869, 38.24233578175403)

## XGBOOST

In [23]:
xgb_params = {
    'max_depth':5,
    'n_estimators' : 50, 
    'learning_rate' : 0.1,
    'seed' : 0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[12:54:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:54:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:55:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [25]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1 ,seed=0)
m.fit(X, y)

imp = PermutationImportance (m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[13:03:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji__cat', 'param_faktura-vat__cat', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']

In [31]:
len(cat_feats)

151

In [32]:
len(feats)

20

In [34]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[13:37:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13407.229151187508, 47.968383519686824)

In [35]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [36]:
df['param_rok-produkcji__cat'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76])

In [37]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']

In [40]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[13:49:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:49:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:49:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11364.016578827623, 80.05911838700821)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )

In [0]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa__cat', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']

In [44]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[13:56:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9724.786775066808, 84.17027602984261)

In [45]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ' , '')) )

In [0]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']

In [48]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[14:07:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:07:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:07:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9570.7287270162, 87.36868463251476)

## DODANIE ZMIAN DO GITHUBA

In [49]:
cd /content/drive/My Drive/Cloab Notebooks/matrix/matrix_two/dw_matrix_car/

/content/drive/My Drive/Cloab Notebooks/matrix/matrix_two/dw_matrix_car


In [50]:
ls

data/            day2_visualization.ipynb  Day_4_XGBoost.ipynb  README.md
Day1_meta.ipynb  Day3_Model.ipynb          LICENSE


Trzeba przenieść aktualny plik do powyższej ścieżki 

In [0]:
!git config --global user.email "smachnio@wp.pl"
!git config --global user.name "Sylwek"

In [54]:
!git add Day4_XGBoost.ipynb
!git commit -m "add XGBoost"

fatal: pathspec 'Day4_XGBoost.ipynb' did not match any files
On branch master
Your branch is ahead of 'origin/master' by 4 commits.
  (use "git push" to publish your local commits)

Untracked files:
	Day_4_XGBoost.ipynb

nothing added to commit but untracked files present


In [0]:
GITHUB_TOKEN='44d04f6f6c31db28fe53daf3bf0519f13ff6e0b6'

In [0]:
!git push -u origin master